# Import libraries

In [42]:
import torch
from piq import ssim, multi_scale_ssim as ms_ssim
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from torch.nn.functional import one_hot

from src.metrics import *
from src.utils import *
from src.datasets import ExcavatorDataset
from src.config import TRANSFORMER, ROOT, DEVICE
from models.Segmentation import DeepLabV3Model, DeepLabV3PlusModel, PyramidAttentionNetworkModel, UNetModel

# Load dataset

In [43]:
train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', plot=False)
print("Number of train imgs: ", len(train_dataset))
test_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test', plot=False)
print("Number of val imgs: ", len(test_dataset))
cls_colors = train_dataset.class_colors

Number of train imgs:  1782
Number of val imgs:  187


# Test 1. VLAD (PASS)

In [44]:
for i in range(30):
    num_cls= np.random.choice([32, 64, 128, 256])
    feature = np.random.choice(['sift', 'root_sift'])
    dataset_choice = np.random.choice(['train', 'validation'])
    if dataset_choice == 'train':
        dataset = train_dataset
    else:
        dataset = test_dataset
    print("Chosen dataset:", dataset_choice)
    idx_1 = 0
    idx_2 = 0
    while idx_1 == idx_2:
        idx_1 = random.randint(0, len(dataset)-1)
        idx_2 = random.randint(0, len(dataset)-1)

    image_1, *_ ,path1= dataset[idx_1]
    image_2, *_, path2 = dataset[idx_2]
    k_means = load_model(f'{ROOT}/models/pickle_model_files/k_means_model_k{num_cls}_{feature}.pkl')

    vlad_1 = VLAD(image_1, k_means=k_means, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(vlad_1))
    vlad_2 = VLAD(image_2, k_means=k_means, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(vlad_2))

    print("VLAD cosine similarity:", res:=cosine_similarity(vlad_1.reshape(1, -1), vlad_2.reshape(1, -1))[0][0])

    hdf_file = load_hdf5(f'{ROOT}/res/vlad/{dataset_choice}/k_means_model_k{num_cls}_{feature}.h5')

    vlad_check_1 = hdf_file[os.path.basename(path1)]
    vlad_check_2 = hdf_file[os.path.basename(path2)]
    print("Check VLAD cosine similarity:", res_check:=cosine_similarity(vlad_check_1.reshape(1, -1), vlad_check_2.reshape(1, -1))[0][0])
    if not abs(res - res_check) < 1e-5:
        raise ValueError("VLAD cosine similarity is not equal to the check value")

print("Test passed")

Chosen dataset: validation
Length of VLAD vector: 32768
Length of VLAD vector: 32768
VLAD cosine similarity: 0.010013854973172413
Check VLAD cosine similarity: 0.010013508588492271
Chosen dataset: validation
Length of VLAD vector: 8192
Length of VLAD vector: 8192
VLAD cosine similarity: -0.05469634852332019
Check VLAD cosine similarity: -0.05469632527270779
Chosen dataset: validation
Length of VLAD vector: 16384
Length of VLAD vector: 16384
VLAD cosine similarity: -0.02144820700178885
Check VLAD cosine similarity: -0.021447399158210555
Chosen dataset: train
Length of VLAD vector: 4096
Length of VLAD vector: 4096
VLAD cosine similarity: -0.07209042410709911
Check VLAD cosine similarity: -0.07209075990486599
Chosen dataset: train
Length of VLAD vector: 32768
Length of VLAD vector: 32768
VLAD cosine similarity: 0.034421698682978084
Check VLAD cosine similarity: 0.03442237937789109
Chosen dataset: validation
Length of VLAD vector: 8192
Length of VLAD vector: 8192
VLAD cosine similarity: -0

# 2. Fisher Vector (PASS)

In [45]:
for i in range(30):
    num_cls= np.random.choice([32, 64, 128, 256])
    feature = np.random.choice(['sift', 'root_sift'])
    dataset_choice = np.random.choice(['train', 'validation'])
    if dataset_choice == 'train':
        dataset = train_dataset
    else:
        dataset = test_dataset
    print("Chosen dataset:", dataset_choice)
    idx_1 = 0
    idx_2 = 0
    while idx_1 == idx_2:
        idx_1 = random.randint(0, len(dataset)-1)
        idx_2 = random.randint(0, len(dataset)-1)

    image_1, *_ ,path1= dataset[idx_1]
    image_2, *_, path2 = dataset[idx_2]
    gmm = load_model(f'{ROOT}/models/pickle_model_files/gmm_model_k{num_cls}_{feature}.pkl')

    fisher_1 = FisherVector(image_1, gmm=gmm, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(fisher_1))
    fisher_2 = FisherVector(image_2, gmm=gmm, flatten=True, feature=feature, verbose=False).vector
    print("Length of VLAD vector:", len(fisher_2))

    print("Fisher cosine similarity:", res:=cosine_similarity(fisher_1.reshape(1, -1), fisher_2.reshape(1, -1))[0][0])

    hdf_file = load_hdf5(f'{ROOT}/res/fisher/{dataset_choice}/gmm_model_k{num_cls}_{feature}.h5')

    fisher_check_1 = hdf_file[os.path.basename(path1)]
    fisher_check_2 = hdf_file[os.path.basename(path2)]
    print("Check Fisher cosine similarity:", res_check:=cosine_similarity(fisher_check_1.reshape(1, -1), fisher_check_2.reshape(1, -1))[0][0])
    if not abs(res - res_check) < 1e-5:
        raise ValueError("Fisher cosine similarity is not equal to the check value")

print("Test passed")


Chosen dataset: validation
Length of VLAD vector: 16448
Length of VLAD vector: 16448
Fisher cosine similarity: -0.01984168187368845
Check Fisher cosine similarity: -0.01984087333947823
Chosen dataset: train
Length of VLAD vector: 32896
Length of VLAD vector: 32896
Fisher cosine similarity: 0.03493780290935161
Check Fisher cosine similarity: 0.03493776378942534
Chosen dataset: validation
Length of VLAD vector: 8224
Length of VLAD vector: 8224
Fisher cosine similarity: -0.0334784002170272
Check Fisher cosine similarity: -0.033479320533289746
Chosen dataset: validation
Length of VLAD vector: 16448
Length of VLAD vector: 16448
Fisher cosine similarity: 0.007157384897983862
Check Fisher cosine similarity: 0.007157188929386087
Chosen dataset: validation
Length of VLAD vector: 8224
Length of VLAD vector: 8224
Fisher cosine similarity: 0.1378242889372241
Check Fisher cosine similarity: 0.1378242537733868
Chosen dataset: train
Length of VLAD vector: 65792
Length of VLAD vector: 65792
Fisher cos

# 3. SSIM, train vs train (PASS)

In [52]:
for i in range(30):
    idx_1 = 0
    idx_2 = 0
    while idx_1 == idx_2:
        idx_1 = random.randint(0, len(train_dataset)-1)
        idx_2 = random.randint(0, len(train_dataset)-1)

    sigma = np.random.choice([0, 2, 4, 6, 8, 10])
    print(f"Chosen sigma for i={i}: ", sigma)
    image_1, *_ ,path1= train_dataset[idx_1]
    print(path1)
    image_2, *_, path2 = train_dataset[idx_2]

    image_1 = TRANSFORMER(image_1)
    image_2 = TRANSFORMER(image_2)

    if sigma != np.array(0):
        sigma = float(sigma)
        image_1 = gaussian_blur(image_1, sigma=sigma, kernel_size = 2*int(3*sigma)+1)
        image_2 = gaussian_blur(image_2, sigma=sigma, kernel_size = 2*int(3*sigma)+1)

    ssim_score = float(SSIM(image_1, image_2).value.item())
    ms_ssim_score = float(MS_SSIM(image_1, image_2).value.item())

    hdf_file = load_hdf5(f'{ROOT}/res/ssim/within_train/ssim_sigma{int(sigma)}.h5')

    path_1_check = None
    idx_arr_1 = None
    path_2_check = None
    idx_arr_2 = None

    for i, path in enumerate(hdf_file['image_paths']):
        if isinstance(path, bytes):
            path = path.decode('utf-8')
        if os.path.basename(path1) == os.path.basename(path):
            idx_arr_1 = i
        if os.path.basename(path2) == os.path.basename(path):
            idx_arr_2 = i

    ssim_check = hdf_file['ssim'][idx_arr_2]
    ms_ssim_check = hdf_file['ms_ssim'][idx_arr_2]
    print("Check SSIM:", ssim_res_check:=ssim_check[idx_arr_1])
    print("Check MS-SSIM:", ms_ssim_res_check:=ms_ssim_check[idx_arr_1])
    if not abs(ssim_score - ssim_res_check) < 1e-5:
       raise ValueError("SSIM is not equal to the check value")
    if not abs(ms_ssim_score - ms_ssim_res_check) < 1e-5:
         raise ValueError("MS-SSIM is not equal to the check value")

print("Test passed")

Chosen sigma for i=0:  10
C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/train\caterpillar\002609_jpg.rf.db40d593ae9ac9af29e2ce07b142d7ab.jpg
Check SSIM: 0.56657505
Check MS-SSIM: 0.3748259
Chosen sigma for i=1:  10
C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/train\roller\4801_png_jpg.rf.9f14a0966b9442f20105061db22a3d8c.jpg
Check SSIM: 0.60616815
Check MS-SSIM: 0.41123337
Chosen sigma for i=2:  10
C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/train\truck\a5_mp4-10_jpg.rf.73592bb1c6bb41a71adf12b35dcecceb.jpg
Check SSIM: 0.46656868
Check MS-SSIM: 0.33169892
Chosen sigma for i=3:  6
C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/train\truck\6f299c89a8604221afb487c8b2a00996_jpeg_jpg.rf.58b4dd26f7fa49b4230be77ad5e32166.jpg
Check SSIM: 0.41617018
Check MS-SSIM: 0.28312802
Chosen sigma for i=4:  10
C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/train\truck\002800_jpg.rf.33338280776f1dc

## 4. SSIM, train vs val

In [53]:
for i in range(30):
    idx_train = 0
    idx_test = 0
    while idx_train == idx_test:
        idx_train = random.randint(0, len(train_dataset)-1)
        idx_test = random.randint(0, len(test_dataset)-1)

    sigma = np.random.choice([0, 2, 4, 6, 8, 10])
    kernel_size = 2*int(3*sigma)+1
    print("Resulting kernel size:", kernel_size)
    print(f"Chosen sigma for i={i}: ", sigma)
    image_train, *_ ,path_train= train_dataset[idx_train]
    image_test, *_, path_test = test_dataset[idx_test]
    print("Chosen image in train dataset:", path_train)
    print("Chosen image in test dataset:", path_test)

    image_train = TRANSFORMER(image_train)
    image_test = TRANSFORMER(image_test)

    if sigma != np.array(0):
        image_train = gaussian_blur(image_train, sigma=float(sigma), kernel_size =kernel_size)
        image_test = gaussian_blur(image_test, sigma=float(sigma), kernel_size = kernel_size)

    ssim_score = ssim(image_train.unsqueeze(0), image_test.unsqueeze(0), data_range=1.0)
    print("Computed SSIM:", ssim_score)
    ms_ssim_score = ms_ssim(image_train.unsqueeze(0), image_test.unsqueeze(0), data_range=1.0)
    print("Computed MS-SSIM:", ms_ssim_score)

    hdf_file = load_hdf5(f'{ROOT}/res/ssim/train_vs_val/ssim_sigma{sigma}.h5')
    test_path_from_file = hdf_file['val_paths'][idx_test]
    train_path_from_file = hdf_file['train_paths'][idx_train]
    if test_path_from_file.decode('utf-8') != os.path.basename(path_test): # Paths are saved as bytes
        raise ValueError(f"Image path does not match. Expected: {os.path.basename(path_test)}, Got: {test_path_from_file}")
    if train_path_from_file.decode('utf-8') != os.path.basename(path_train):
        raise ValueError(f"Image path does not match. Expected: {os.path.basename(path_train)}, Got: {train_path_from_file}")
    ssim_check = hdf_file['ssim'][idx_test, idx_train]
    ms_ssim_check = hdf_file['ms_ssim'][idx_test, idx_train]
    print("Check SSIM:", ssim_res_check:=ssim_check)
    print("Check MS-SSIM:", ms_ssim_res_check:=ms_ssim_check)
    if not abs(ssim_score - ssim_res_check) < 1e-5:
       raise ValueError(f"SSIM is not equal to the check value. Expected: {ssim_res_check}, Got: {ssim_score}")
    if not abs(ms_ssim_score - ms_ssim_res_check) < 1e-5:
         raise ValueError(f"MS-SSIM is not equal to the check value. Expected: {ms_ssim_res_check}, Got: {ms_ssim_score}")


Resulting kernel size: 61
Chosen sigma for i=0:  10
Chosen image in train dataset: C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/train\excavator\a3_mp4-7_jpg.rf.c0d545a43601232ca6f45515e9cf628c.jpg
Chosen image in test dataset: C:\workspace\similarity_metrics_of_images/excavator_dataset_w_masks/test\roller\0040094_jpg.rf.dce498bce484ef500e7e67f54fb0850d.jpg


TypeError: 'float' object is not callable

# 5. Model predictions, IoU, all

In [11]:
# DeepLabV3
dlv3 =DeepLabV3Model(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3_HybridFocalDiceLoss.pt')

# DeepLabV3Plus
dlv3p = DeepLabV3PlusModel(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3Plus_HybridFocalDiceLoss.pt')

# UNet
unet = UNetModel(model_path=f'{ROOT}/models/torch_model_files/UNet_HybridFocalDiceLoss.pt')

# Pyramid Attention Network
pan = PyramidAttentionNetworkModel(model_path=f'{ROOT}/models/torch_model_files/PyramidAttentionNetwork_HybridFocalDiceLoss.pt')

2024-12-24 20:26:36,419 - DeepLabV3 - INFO - Device used for model: cuda
2024-12-24 20:26:37,786 - DeepLabV3Plus - INFO - Device used for model: cuda
2024-12-24 20:26:38,374 - UNet - INFO - Device used for model: cuda
2024-12-24 20:26:38,927 - PyramidAttentionNetwork - INFO - Device used for model: cuda


In [13]:
transformed_train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', transform=TRANSFORMER)
print("Number of training samples:", num_train_imgs:=len(train_dataset))
transformed_test_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test', transform=TRANSFORMER)
print("Number of test samples:", num_test_imgs:=len(test_dataset))

Number of training samples: 1782
Number of test samples: 187


In [14]:
def plot_image_and_mask(image: torch.Tensor, gt_mask: torch.Tensor, pred_mask: torch.Tensor=None):
    """
    Plot image, ground truth mask and predicted mask (if available).

    :param image:
    :param gt_mask:
    :param pred_mask:
    :return:
    """
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 3, 1)
    if image.shape[0] == 3:
        image = image.permute(1, 2, 0)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Image')
    plt.subplot(1, 3, 2)
    if gt_mask.shape[0] == 3:
        gt_mask = gt_mask.permute(1, 2, 0)
    plt.imshow(gt_mask)
    plt.axis('off')
    plt.title('Ground Truth Mask')

    if pred_mask is None:
        plt.show()
        return

    plt.subplot(1, 3, 3)
    if pred_mask.shape[0] == 3:
        pred_mask = pred_mask.permute(1, 2, 0)
    plt.imshow(pred_mask)
    plt.axis('off')
    plt.title('Predicted Mask')
    plt.show()

for i in range(30):
    model = np.random.choice([dlv3, dlv3p, unet, pan])
    print("Chosen model:", model.model.__class__.__name__)
    dataset_choice = np.random.choice(['train', 'val'])
    if dataset_choice == 'train':
        chosen_dataset = transformed_train_dataset
        idx = random.randint(0, len(train_dataset)-1)
        print(f"Chosen image in train dataset: {idx}")
    else:
        chosen_dataset = transformed_test_dataset
        idx = random.randint(0, len(test_dataset)-1)
        print(f"Chosen image in val dataset: {idx}")

    image, mask, path = chosen_dataset[idx]
    path_basename = os.path.basename(path)
    print("Chosen image path:", path_basename)

    #_, pred_mask = model.predict_single_image(image, mask, raw_output=False)

    _, raw_pred= model.predict_single_image(image, mask, raw_output=True)
    mask= one_hot(mask.long(), num_classes=12).permute(2, 0, 1).float()
    iou_manual = multiclass_iou(pred_mask=raw_pred, true_mask=mask)
    print("dTYPE of raw_pred:", raw_pred.dtype)
    print("dTYPE of mask:", mask.dtype)
    print("Computed IoU:", iou_manual)

    model_paths = {
        dlv3: f'{ROOT}/res/model_performance/{dataset_choice}_iou_dlv3.h5',
        dlv3p: f'{ROOT}/res/model_performance/{dataset_choice}_iou_dlv3p.h5',
        unet: f'{ROOT}/res/model_performance/{dataset_choice}_iou_unet.h5',
        pan: f'{ROOT}/res/model_performance/{dataset_choice}_iou_pan.h5'
    }

    iou_data = load_hdf5(model_paths[model])[path_basename]

    if not abs(iou_manual - iou_data) < 1e-5:
        raise ValueError(f"IoU difference is not equal to the check value. Manual: {iou_manual}, Check: {iou_data}")

    #plot_image_and_mask(image, mask, pred_mask)
print("All tests passed")

Chosen model: Unet
Chosen image in train dataset: 1368
Chosen image path: 002818_jpg.rf.c02a89d2dd106e4b4d7d532600c18a58.jpg


C:\Users\STUD_VuNhat\AppData\Local\anaconda3\envs\conda_env\Lib\site-packages\torch\nn\modules\module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


dTYPE of raw_pred: torch.float32
dTYPE of mask: torch.float32
Computed IoU: tensor(0.6741)
Chosen model: PAN
Chosen image in train dataset: 79
Chosen image path: 002615_jpg.rf.fa926ea36e23783fd5832515e987a082.jpg
dTYPE of raw_pred: torch.float32
dTYPE of mask: torch.float32
Computed IoU: tensor(0.8547)
Chosen model: Unet
Chosen image in train dataset: 1658
Chosen image path: a5_mp4-1_jpg.rf.b1f762cec1a96fc88a2a6b03dd902191.jpg
dTYPE of raw_pred: torch.float32
dTYPE of mask: torch.float32
Computed IoU: tensor(0.7267)
Chosen model: DeepLabV3Plus
Chosen image in val dataset: 66
Chosen image path: 8b3ef1fba766bd5ccebd25fccb4611b5_jpeg_jpg.rf.1af38b770384ee65728cc9c3e9a54b15.jpg
dTYPE of raw_pred: torch.float32
dTYPE of mask: torch.float32
Computed IoU: tensor(0.9719)
Chosen model: PAN
Chosen image in train dataset: 183
Chosen image path: 002697_jpg.rf.d242dd59c4dbee385b3588a57ab9240e.jpg
dTYPE of raw_pred: torch.float32
dTYPE of mask: torch.float32
Computed IoU: tensor(0.8016)
Chosen model

# 6. Model predictions, IoU (PASS)

In [23]:
for i in range(30):
    model = np.random.choice([dlv3, dlv3p, unet, pan])
    print("Chosen model:", model.model.__class__.__name__)
    dataset_choice = np.random.choice(['train', 'val'])
    background = np.random.choice(['with_bg', 'no_bg'])
    if dataset_choice == 'train':
        chosen_dataset = transformed_train_dataset
        idx = random.randint(0, len(train_dataset)-1)
        print(f"Chosen image in train dataset: {idx}")
    else:
        chosen_dataset = transformed_test_dataset
        idx = random.randint(0, len(test_dataset)-1)
        print(f"Chosen image in val dataset: {idx}")

    image, mask, path = chosen_dataset[idx]
    path_basename = os.path.basename(path)
    print("Chosen image path:", path_basename)

    ignore_background = True if background == 'no_bg' else False
    probs_vector, _ = model.predict_single_image(image, mask, return_raw_prob_vector=True, ignore_background=ignore_background)

    print("Computed Probs vector:", probs_vector)

    model_paths = {
        dlv3: f'{ROOT}/res/confidence_vectors/dlv3_{background}_{dataset_choice}.h5',
        dlv3p: f'{ROOT}/res/confidence_vectors/dlv3p_{background}_{dataset_choice}.h5',
        unet: f'{ROOT}/res/confidence_vectors/unet_{background}_{dataset_choice}.h5',
        pan: f'{ROOT}/res/confidence_vectors/pan_{background}_{dataset_choice}.h5'
    }

    probs_data = load_hdf5(model_paths[model])[path_basename]

    if not torch.allclose(probs_vector, torch.tensor(probs_data, device=DEVICE)):
        raise ValueError(f"Probs vector is not equal to the check value. Manual: {probs_vector}, Check: {probs_data}")

print("All tests passed")


Chosen model: PAN
Chosen image in val dataset: 38
Chosen image path: 002841_jpg.rf.6f4825c54a3b2d3d0c26e1eecfef8272.jpg
Computed Probs vector: tensor([5.9829e-06, 1.6947e-08, 2.2731e-01, 2.0450e-05, 4.4153e-06, 5.1458e-09,
        1.7295e-03, 1.7937e-04, 7.0090e-03, 1.2120e-11, 5.6356e-02],
       device='cuda:0')
Chosen model: PAN
Chosen image in train dataset: 1704
Chosen image path: c84e9cfa8a049bc23b51d37e2e740f7e_jpg.rf.d695d719a9d2ab4c0676fdebb7a8708c.jpg
Computed Probs vector: tensor([8.9804e-01, 3.7576e-07, 1.9948e-07, 2.3155e-03, 3.4147e-02, 5.7239e-07,
        7.8097e-09, 3.2144e-05, 9.2584e-04, 7.7372e-06, 3.0307e-07, 6.4531e-02],
       device='cuda:0')
Chosen model: DeepLabV3Plus
Chosen image in train dataset: 769
Chosen image path: a3_mp4-7_jpg.rf.2887e2a33cca3b73ea107d3f2277dff1.jpg
Computed Probs vector: tensor([6.9596e-01, 1.3501e-04, 2.6570e-05, 3.1044e-02, 4.3977e-02, 4.4982e-05,
        4.8062e-06, 1.5270e-01, 5.2704e-06, 3.2799e-05, 8.2243e-07, 7.6062e-02],
       

# 7. Model predictions, IoU, Only classes contained in train mask (PASS)

In [37]:
for i in range(30):
    model = np.random.choice([dlv3, dlv3p, unet, pan])
    print("Chosen model:", model.model.__class__.__name__)
    dataset_choice = np.random.choice(['train', 'val'])
    chosen_dataset = transformed_train_dataset


    cls_colors = chosen_dataset.class_colors
    image, mask, path = chosen_dataset[idx]
    path_basename = os.path.basename(path)
    print("Chosen image path:", path_basename)
    classes_of_interest = get_class_idx(mask, cls_colors)

    probs_vector, _ = model.predict_single_image(image, mask, return_raw_prob_vector=True, ignore_background=False, cls_of_interest=classes_of_interest)

    print("Computed Probs vector:", probs_vector)

    model_paths = {
        dlv3: f'{ROOT}/res/confidence_vectors/dlv3_only_classes_in_train.h5',
        dlv3p: f'{ROOT}/res/confidence_vectors/dlv3p_only_classes_in_train.h5',
        unet: f'{ROOT}/res/confidence_vectors/unet_only_classes_in_train.h5',
        pan: f'{ROOT}/res/confidence_vectors/pan_only_classes_in_train.h5'
    }

    probs_data = load_hdf5(model_paths[model])[path_basename]
    classes = probs_data['classes']
    condifences = probs_data['confidence'][classes]

    if not torch.allclose(probs_vector, torch.tensor(condifences, device=DEVICE).to(torch.float32)):
        raise ValueError(f"Probs vector is not equal to the check value. Manual: {probs_vector}, Check: {probs_data}")

    if not all(classes == classes_of_interest):
        raise ValueError(f"Classes are not equal to the check value. Manual: {classes_of_interest}, Check: {classes}")

print("All tests passed")

Chosen model: DeepLabV3Plus
Chosen image path: 002645_jpg.rf.d766d9edb0cc2c5565d5e47dbfd49fb0.jpg
Computed Probs vector: tensor([0.7950, 0.1612], device='cuda:0')
Chosen model: DeepLabV3Plus
Chosen image path: 002645_jpg.rf.d766d9edb0cc2c5565d5e47dbfd49fb0.jpg
Computed Probs vector: tensor([0.7950, 0.1612], device='cuda:0')
Chosen model: DeepLabV3
Chosen image path: 002645_jpg.rf.d766d9edb0cc2c5565d5e47dbfd49fb0.jpg
Computed Probs vector: tensor([0.8958, 0.0855], device='cuda:0')
Chosen model: PAN
Chosen image path: 002645_jpg.rf.d766d9edb0cc2c5565d5e47dbfd49fb0.jpg
Computed Probs vector: tensor([0.9212, 0.0505], device='cuda:0')
Chosen model: PAN
Chosen image path: 002645_jpg.rf.d766d9edb0cc2c5565d5e47dbfd49fb0.jpg
Computed Probs vector: tensor([0.9212, 0.0505], device='cuda:0')
Chosen model: Unet
Chosen image path: 002645_jpg.rf.d766d9edb0cc2c5565d5e47dbfd49fb0.jpg
Computed Probs vector: tensor([0.8258, 0.0483], device='cuda:0')
Chosen model: DeepLabV3
Chosen image path: 002645_jpg.r

In [32]:
probs_vector

tensor([0.7950, 0.1612], device='cuda:0')